# 圖書館課程爬蟲

## 怎麼知道有幾頁?  

課程首頁  
http://web.lib.fcu.edu.tw/ilworkshop/taxonomy/term/5  

課程頁面  
http://web.lib.fcu.edu.tw/ilworkshop/taxonomy/term/5?page=1  

In [2]:
import requests
from bs4 import Tag, BeautifulSoup as bs
import re
# 莊文明
def get_page_num(link):
    r = requests.get(link)
    soup = bs(r.text, "lxml")
    content = soup.find('a', {'title': '到最後一頁'},href=True).get('href')
    page_num = int(re.findall(r"[0-9]+",content)[1])

    return page_num

page_num = get_page_num('http://web.lib.fcu.edu.tw/ilworkshop/taxonomy/term/5')
page_link = 'http://web.lib.fcu.edu.tw/ilworkshop/taxonomy/term/5?page={0}'
page_num

49

## 有那些重要欄位?  
http://web.lib.fcu.edu.tw/ilworkshop/workshops/20150702-146.html

In [3]:
row_title = ['tag1','tag2','課程編號', '主題', '講者', '開始時間','結束時間', '總計分鐘數', 
             '地點', '適合對象', '限制人數', '已報名人數','已爆滿','描述','講義檔案名稱','講義網址','網址']  # 欄位標題

tag1_f = []
tag2_f = []
title_f = []
link_f = []
start_date_f = []
end_date_f = []
description_f = []
audience_f = []

speaker_f = []
id_f = []
time_m_f = []
location_f = []
limit_f = []
register_f = []
full_f = []
file_name_f = []
file_link_f = []

## 如何抓取頁面欄位

http://web.lib.fcu.edu.tw/ilworkshop/taxonomy/term/5?page=1  
tag, 主題, 開始時間, 結束時間, 適合對象, 描述, 網址  

http://web.lib.fcu.edu.tw/ilworkshop/workshops/20150209-107.html  
課程編號, 講者, 總計分鐘數, 地點, 限制人數, 已報名人數, 已爆滿, 講義  

In [4]:
import requests
from bs4 import Tag, BeautifulSoup as bs
import re

In [5]:
def get_content(link):
    print(link)
    id = ''
    speaker = ''
    time_m = ''
    location = ''
    limiit_num = ''
    register = ''
    full = ''
    file_name = ''
    file_link = ''

    r = requests.get(link)
    soup = bs(r.text, "lxml")
    content = soup.find('div', {'class': 'region region-content'})
    id = content.find('div', {'class': 'field field-name-field-workshop-id field-type-text field-label-inline clearfix'}).find('div', {'class': 'field-item even'}).text.strip()
    speaker = content.find('div', {'class': 'field field-name-field-workshop-speaker field-type-text field-label-inline clearfix'}).find('div', {'class': 'field-item even'}).text.strip()
    time = content.find('div', {'class': 'field field-name-field-workshop-times field-type-number-integer field-label-inline clearfix'})
    if time != None:
        time = time.find('div', {'class': 'field-item even'}).text.strip()
        time_m = int(re.findall(r"[0-9]+", time)[0])

    location = content.find('div', {'class': 'field field-name-field-workshop-location field-type-list-text field-label-inline clearfix'})
    if location != None:
        location = location.find('div', {'class': 'field-item even'}).text.strip()

    limit = content.find('div', {'class': 'field field-name-field-workshop-limit field-type-number-integer field-label-inline clearfix'})
    if limit != None:
        limit = limit.find('div', {'class': 'field-item even'}).text.strip()
        nums = re.findall(r"[0-9]+",limit)
        limiit_num = int(nums[0])
        if '已額滿' in limit:
            register = limiit_num
            full = 1
        else:
            register = int(nums[1])
            full = 0
    file = content.find('div', {'class': 'field field-name-field-workshop-dlfile field-type-file field-label-above'})
    if file != None:
        file_name = file.find('a', href=True).get('title')
        file_link = file.find('a', href=True).get('href')

    return id,speaker,time_m,location,limiit_num,register,full,file_name,file_link

In [6]:
def get_view_content(link):
    r = requests.get(link)
    soup = bs(r.text, "lxml")
    view_content = soup.find('div',{'class':'view-content'})

    for child in view_content.children:
        if isinstance(child, Tag):
            tags = child.find('div', {'class': 'tags'})
            tag1 = tags.find_all('a', href=True)[0].text.strip()
            tag2 = tags.find_all('a', href=True)[1].text.strip()
            tag1_f.append(tag1)
            tag2_f.append(tag2)

            title = child.find('h3', {'class': 'title'})
            title_f.append(title.text.strip())

            link = 'http://web.lib.fcu.edu.tw/' + title.find('a', href=True).get('href')
            link_f.append(link)

            start = child.find('span', {'class': 'date-display-start'}).get('content').strip()
            end = child.find('span', {'class': 'date-display-end'}).get('content').strip()
            start_date_f.append(start)
            end_date_f.append(end)

            desc = child.find('div', {'class': 'description'})
            if desc != None:
                description_f.append(desc.text.strip())
            else:
                description_f.append('')

            audience = child.find('div', {'class': 'participants'}).text.replace('適合對象: ','').strip()
            audience_f.append(audience)

            id, speaker, time,location, limilt_num, register, full,file_name,file_link = get_content(link)
            id_f.append(id)
            speaker_f.append(speaker)
            time_m_f.append(time)
            location_f.append(location)
            limit_f.append(limilt_num)
            register_f.append(register)
            full_f.append(full)
            file_name_f.append(file_name)
            file_link_f.append(file_link)

In [7]:
for n in range(page_num-48):
    print('page ',n)
    link = page_link.format(n)
    get_view_content(link)

page  0
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150702-146.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150701-144.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150701-143.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150629-142.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150629-141.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150629-140.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150629-139.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150624-137.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150624-136.html
http://web.lib.fcu.edu.tw//ilworkshop/workshops/20150530-132.html


In [8]:
import pandas as pd

# 資料儲存成 dataframe
print(row_title)
#row_title = ['tag1','tag2','課程編號', '講者', '開始時間','結束時間', '總計分鐘數', '地點', '適合對象', '限制人數', '已報名人數','已爆滿','描述','網址']  # 欄位標題
library_df = pd.DataFrame({ row_title[0]:tag1_f,
                            row_title[1]:tag2_f,
                            row_title[2]:id_f,
                            row_title[3]:title_f,
                            row_title[4]:speaker_f,
                            row_title[5]:start_date_f,
                            row_title[6]:end_date_f,
                            row_title[7]:time_m_f,
                            row_title[8]:location_f,
                            row_title[9]:audience_f,
                            row_title[10]:limit_f,
                            row_title[11]:register_f,
                            row_title[12]: full_f,
                            row_title[13]: description_f,
                            row_title[14]: file_name_f,
                            row_title[15]: file_link_f,
                            row_title[16]: link_f})

library_df.head()
writer = pd.ExcelWriter('library.xlsx',options={'strings_to_urls': True})
library_df.to_excel(writer, 'workshop',index=False,encoding="utf-8")
writer.save()

['tag1', 'tag2', '課程編號', '主題', '講者', '開始時間', '結束時間', '總計分鐘數', '地點', '適合對象', '限制人數', '已報名人數', '已爆滿', '描述', '講義檔案名稱', '講義網址', '網址']
